# Chapter 1. Hello Transformers

Em 2017, pesquisadores do Google publicaram um artigo que propunha uma nova arquitetura de rede neural para modelagem de sequência. Apelidada de *Transformer*, essa arquitetura superou as redes neurais recorrentes (RNNs) em tarefas de tradução automática, tanto em termos de qualidade de tradução quanto de custo de treinamento.

**Nota:** A. Vaswani et al., “Attention Is All You Need”, (2017). This title was so catchy that no less than 50 follow-up papers have included “all you need” in their titles!

Link: https://arxiv.org/abs/1706.03762

Em paralelo, um método eficaz de aprendizado de transferência chamado ULMFiT mostrou que o treinamento de redes de memória de longo prazo (LSTM) em um corpus muito grande e diversificado poderia produzir classificadores de texto de última geração com poucos dados rotulados.

Link:  J. Howard and S. Ruder, “Universal Language Model Fine-Tuning for Text Classification”, (2018).

https://arxiv.org/abs/1801.06146

Esses avanços foram os catalisadores para dois dos transformadores mais conhecidos da atualidade: o Generative Pretrained Transformer (GPT)3 e o Bidirectional Encoder Representations from Transformers (BERT).

Link: A. Radford et al., “Improving Language Understanding by Generative Pre-Training”, (2018).

https://openai.com/blog/language-unsupervised/


J. Devlin et al., “BERT: Pre-Training of Deep Bidirectional Transformers for Language Understanding”, (2018).

https://arxiv.org/abs/1810.04805

Ao combinar a arquitetura *Transformer* com aprendizado não supervisionado, esses modelos eliminaram a necessidade de treinar arquiteturas específicas de tarefas do zero e quebraram quase todos os benchmarks em PNL por uma margem significativa. Desde o lançamento do GPT e do BERT, surgiu um zoológico de modelos de transformadores.

<img src=fig-1-1.PNG>

Para entender o que há de novo nos transformadores, primeiro precisamos explicar:

* The encoder-decoder framework

* Attention mechanisms

* Transfer learning

Neste capítulo, será abordado os principais conceitos que fundamentam a difusão dos transformadores, terá um tour por algumas das tarefas em que eles se destacam e concluirá com uma olhada no ecossistema de ferramentas e bibliotecas do Hugging Face.

## The Encoder-Decoder Framework - (A estrutura do codificador-decodificador)

Antes dos transformadores, arquiteturas recorrentes como LSTMs eram o estado da arte em PNL. Essas arquiteturas contêm um loop de feedback nas conexões de rede que permitem que as informações se propaguem de uma etapa para outra, tornando-as ideais para modelar dados sequenciais como texto.

Conforme ilustrado no lado esquerdo da figura 1-2, um RNN recebe alguma entrada (que pode ser uma palavra ou caractere), a alimenta pela rede e emite um vetor chamado estado oculto.

Conforme imagem abaixo, um RNN recebe alguma entrada (que pode ser uma palavra ou caractere), a alimenta pela rede e gera um vetor chamado *hidden state* (estado oculto). Ao mesmo tempo, o modelo alimenta algumas informações de volta para si mesmo por meio do ciclo de *feedback*, que pode ser usado na próxima etapa. Isso pode ser visto com mais clareza se “unroll” o loop conforme mostrado no lado direito, o RNN passa informações sobre seu estado em cada etapa para a próxima operação na sequência.

Isso permite que uma RNN acompanhe as informações das etapas anteriores e as use para suas previsões de saída.

<img src=fig-1-2.PNG>

**Nota:** Essas arquiteturas foram (e continuam sendo) amplamente utilizadas para tarefas de PNL, processamento de fala e séries temporais.

**Indicação de leitura:** https://karpathy.github.io/2015/05/21/rnn-effectiveness/


Uma área em que as RNNs tiveram um papel importante foi no desenvolvimento de sistemas de tradução automática, onde o objetivo é mapear uma sequência de palavras de um idioma para outro.

Esse tipo de tarefa geralmente é realizado com um codificador-decodificador ou arquitetura sequência-a-sequência, que é adequada para situações em que a entrada e a saída são ambas sequências de comprimento arbitrário.

O trabalho do codificador é codificar as informações da sequência de entrada em uma representação numérica que geralmente é chamada de  *last hidden state* (último estado oculto). Este estado é então passado para o decodificador, que gera a sequência de saída.

Em geral, os componentes do codificador e do decodificador podem ser qualquer tipo de arquitetura de rede neural que possa modelar sequências.

A imagem mostra que para um par de RNNs, onde a frase em inglês "Transformers are great!" é codificado como um vetor de estado oculto que é então decodificado para produzir a tradução alemã “Transformer sind grossartig!”. As palavras de entrada são alimentadas sequencialmente através do codificador e as palavras de saída são geradas uma de cada vez, de cima para baixo.

<img src=fig-1-3.PNG>

Embora elegante em sua simplicidade, uma fraqueza dessa arquitetura é que o estado oculto final do codificador cria um *information bottleneck* (gargalo de informações): ele tem que representar o significado de toda a sequência de entrada porque isso é tudo a que o decodificador tem acesso ao gerar a saída. Isso é especialmente desafiador para sequências longas, em que as informações no início da sequência podem ser perdidas no processo de compactação de tudo em uma representação única e fixa.

Felizmente, existe uma saída para esse gargalo, permitindo que o decodificador tenha acesso a todos os estados ocultos do codificador. O mecanismo geral para isso é chamado de *attention* e é um componente-chave em muitas arquiteturas de redes neurais modernas.

Entender como a *attention* foi desenvolvida para RNNs nos colocará em boa forma para entender um dos principais blocos de construção da arquitetura do *Transformer*.


## Attention Mechanisms

A ideia principal por trás da *attention* é que, em vez de produzir um único estado oculto para a sequência de entrada, o codificador emite um estado oculto em cada etapa que o decodificador pode acessar. No entanto, usar todos os estados ao mesmo tempo criaria uma enorme entrada para o decodificador, portanto, algum mecanismo é necessário para priorizar quais estados usar.

É aqui que entra a atenção: permite que o decodificador atribua uma quantidade diferente de peso, ou “attention”, a cada um dos estados do codificador em cada passo de tempo de decodificação.

Esse processo é ilustrado na Figura 1-4, onde o papel da *attention* é mostrado para prever o terceiro token na sequência de saída.

<img src=fig-1-4.PNG>

Concentrando-se em quais *tokens* de entrada são mais relevantes em cada passo de tempo, esses modelos baseados em *attention* são capazes de aprender alinhamentos não triviais entre as palavras em uma tradução gerada e aquelas em uma frase de origem.

Por exemplo, a Figura 1-5 visualiza os pesos de *attention* para um modelo de tradução de inglês para francês, em que cada pixel denota um peso. A figura mostra como o decodificador consegue alinhar corretamente as palavras “zone” e “Area”, que são ordenadas de forma diferente nos dois idiomas.

<img src=fig-1-5.PNG>

Embora a *attention* tenha possibilitado a produção de traduções muito melhores, ainda havia uma grande deficiência no uso de modelos recorrentes para o codificador e decodificador: os cálculos são inerentemente sequenciais e não podem ser paralelizados na sequência de entrada.

Com o *transformer*, um novo paradigma de modelagem foi introduzido: dispensar totalmente a recorrência e, em vez disso, confiar inteiramente em uma forma especial de atenção chamada *self-attention*. Abordaremos a *self-attention* com mais detalhes no Capítulo 3, mas a ideia básica é permitir que a *attention* opere em todos os estados na mesma camada da rede neural.

Isso é mostrado na Figura 1-6, onde o codificador e o decodificador têm seus próprios mecanismos de *self-attention*, cujas saídas são alimentadas para redes neurais de alimentação direta (FF NNs).

Essa arquitetura pode ser treinada muito mais rapidamente do que os modelos recorrentes e abriu o caminho para muitos dos avanços recentes em NLP.

<img src=fig-1-6.PNG>

**Nota:** No artigo original do Transformer, o modelo de tradução foi treinado do zero em um grande corpus de pares de frases em vários idiomas. No entanto, em muitas aplicações práticas da NLP, não temos acesso a grandes quantidades de dados de texto rotulados para treinar nossos modelos. Faltava uma peça final para iniciar a revolução do transformador: o *transfer learning* (aprendizado de transferência).